This dataset is from Kaggle (Links to an external site.) and contains employee metrics such as satisfaction level, number of projects completed, time spent at the company, and number of promotions. In a hypothetical scenario, we could use this data to determine whether or not an employee is eligible for a bonus. This dataset has four variables, each using different numerical scales and units. If we want to use this dataset for a neural network model, we need to standardize these variables, otherwise it is likely that "Satisfaction_Level" (which ranges between 0 and 1) would be undervalued compared to "Time_Spent" (which ranges from approximately 50 to 4,800 hours). 

In [2]:
#dependencies
import pandas as pd
from sklearn.preprocessing import StandardScaler

#read data
hrDF = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/hr_dataset.csv")
hrDF.head()

,Satisfaction_Level,Num_Projects,Time_Spent,Num_Promotions
0,0.30,1,253,2
1,0.25,1,200,0
2,0.90,4,2880,5
3,0.65,3,1450,3
4,0.50,2,785,2


To apply our standardization, we need to create a StandardScaler instance and then fit the input data. After fitting, we can transform and standardize the dataset. Lastly, once we have our transformed data within the StandardScaler instance, we must export the transformed data into a Pandas DataFrame. 

In [5]:
#scaler instance
scaler=StandardScaler()

#fit scaler
scaler.fit(hrDF)

#xform/standardarize dataset
scaledData = scaler.transform(hrDF)

#create dataframe of scaled data
xformed_scaled_data = pd.DataFrame(scaledData, columns=hrDF.columns)
xformed_scaled_data.head()

,Satisfaction_Level,Num_Projects,Time_Spent,Num_Promotions
0,-1.303615,-1.162476,-1.049481,-0.558656
1,-1.512945,-1.162476,-1.094603,-1.804887
2,1.208335,0.860233,1.187080,1.310692
3,0.161689,0.185996,-0.030385,0.064460
4,-0.466299,-0.488240,-0.596549,-0.558656


Looking at the results of our transformation, all of the variables have now been standardized, with a mean value of 0 and a standard deviation of 1. Now the data is ready to be passed along to our neural network model.

Neural networks are designed to calculate weights for various input data and pass along an output value based on an activation function. With our basic neural networks, input data is parsed using an input layer, evaluated in a single hidden layer, then calculated in the output layer. In other words, a basic neural network is designed such that the input values are evaluated *only once* before they are used in a classification or regression equation. Although basic neural networks are relatively easy to conceptualize and understand, there are limitations to using a basic neural network, such as:

    - A basic neural network with many neurons will require more training data than other comparable statistics and machine learning models to produce an adequate model.
    - Basic neural networks struggle to interpret complex nonlinear numerical data, or data with many confounding factors that have hidden effects on more than one variable.
    - Basic neural networks are incapable of analyzing image datasets without severe data preprocessing.

To address the limitations of the basic neural network, we can implement a more robust neural network model by adding additional hidden layers. A neural network with more than one hidden layer is known as a **deep neural network**:


Deep neural networks function similarly to the basic neural network, with one major exception. The outputs of one hidden layer of neurons (that have been evaluated and transformed using an activation function) become the inputs to additional hidden layers of neurons. As a result, the next layer of neurons can evaluate higher order interactions between weighted variables and identify complex, nonlinear relationships across the entire dataset. These additional layers can observe and weight interactions between clusters of neurons across the entire dataset, which means they can identify and account for more information than any number of neurons in a single hidden layer.

Deep neural network models also are commonly referred to as **deep learning models** due to their ability to learn from example data, regardless of the complexity or data input type. Just like humans, deep learning models can identify patterns, determine severity, and adapt to changing input data from a wide variety of data sources. Compared to basic neural network models, which require a large number of neurons to identify nonlinear characteristics, deep learning models only need a few neurons across a few hidden layers to identify the same nonlinear characteristics.

For our first deep learning example, we'll look at another human resources (HR) dataset from [IBM's HR department](https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/HR-Employee-Attrition.csv) (Links to an external site.). This dataset contains employee career profile and attrition (departure from company) information, such as age, department, job role, work/life balance, and so forth. Using this data, we can generate a deep learning model that can help to identify whether or not a person is likely to depart from the company given his or her current employee profile.

This dataset has more than 30 variables, each with different numerical distributions and categorical variables, as well as some dichotomous variables that will need transformation.

This will be the deeplearning tabular portion.

In [6]:
#import other dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

#import dataset

attritionDF = pd.read_csv("https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-online/module_19/HR-Employee-Attrition.csv")
attritionDF.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


Looking at the top of our DataFrame, there are multiple columns with categorical values as well as our numerical values. To make things easier, we should generate a list of categorical variable names. Instead of searching across all 35 columns and keeping track of which variables need categorical preprocessing, we'll let Python do all of the heavy lifting. As an added bonus, we can use our variable list to perform the one-hot encoding once, rather than for each individual variable. To generate our variable list, we'll use Pandas Dataframe.dtypes property. 

In [9]:
#generate categorical variable list
attrCat = attritionDF.dtypes[attritionDF.dtypes == "object"].index.tolist()
attrCat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

Now that we have our variable names separated, we can start to preprocess our data starting with the one-hot encoding of the categorical data. Looking at our `attrition_cat` variable, there are eight categorical variables that need encoding. However, before we loop through our variables and encode them using Scikit-learn's `OneHotEncoder` module, we need to make sure that none of the categorical variables have more than 10 unique values and require bucketing.

In [10]:
#chekc number of unique variables
attritionDF[attrCat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

According to the `nunique` method, none of the categorical variables have more than 10 unique values, which means we're ready to encode. Previously, we have encoded a single variable at a time, but Scikit-learn is flexible enough to perform all of the one-hot encodings at the same time. The only difference from our single variable example is we need to pass our `attrition_cat` variable list.

In [11]:
#create onehotencoder instance
enc = OneHotEncoder(sparse=False)

#fit/xform
encDF = pd.DataFrame(enc.fit_transform(attritionDF[attrCat]))

#add names to df
encDF.columns = enc.get_feature_names_out(attrCat)
encDF.head()


,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


Now that our categorical variables have been encoded, they are ready to replace our unencoded categorical variables in our dataset. To replace these columns, we'll use a combination of Pandas' `merge` and `drop` methods.

In [13]:
#merge onehot features and drop originals
attritionDF = attritionDF.merge(encDF, left_index=True, right_index=True).drop(attrCat,1)
attritionDF.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


o build our training and testing datasets, we need to separate two values:

- input values (which are our independent variables commonly referred to as **model features** or "X" in [TensorFlow documentation](https://www.tensorflow.org/guide/))
- target output (our dependent variable commonly referred to as **target** or "y" in TensorFlow documentation)

For our purposes, we want to build a model that will predict whether or not a person is at risk for attrition; therefore, we must separate the "Attrition" columns from the rest of the input data. In fact, because the attrition data is dichotomous (one of two values), we only need to keep the "Attrition_Yes" column—we can ignore the "Attrition_No" column because it is redundant.

In [19]:
# Split our preprocessed data into our features and target arrays
y = attritionDF["Attrition_Yes"].values
X = attritionDF.drop(["Attrition_Yes", "Attrition_No"],1).values


# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=78)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


Now that our training and testing data have been allocated, we're ready to build our StandardScalerobject and standardize the numerical features, which again means that all values become get scaled to fall within one standard deviation of the mean, which becomes 0

In [20]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Now that our data is preprocessed via one-hot encoding and standardization, we should probably perform a gut check to ensure that no data has been lost from our original DataFrame.  t last, our data is preprocessed and separated and ready for modelling. For our purposes, we will use the same framework we used for our basic neural network:

- For our input layer, we must add the number of input features equal to the number of variables in our feature DataFrame.
- In our hidden layers, our deep learning model structure will be slightly different—we'll add two hidden layers with only a few neurons in each layer. To create the second hidden layer, we'll add another Keras Dense class while defining our model. All of our hidden layers will use the relu activation function to identify nonlinear characteristics from the input values.
- In the output layer, we'll use the same parameters from our basic neural network including the sigmoid activation function. The sigmoid activation function will help us predict the probability that an employee is at risk for attrition.


In [24]:
# Define the model - deep neural net
inputFeatures = len(X_train[0])
hiddenNodes_layer1 = 8
hiddenNodes_layer2 = 5

#instantiate neural network keras model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hiddenNodes_layer1, input_dim=inputFeatures, activation="relu")
)

# Second hidden layer, don't have to pass the input_dim argument because it is part of the first layer
nn.add(
    tf.keras.layers.Dense(units=hiddenNodes_layer2, activation="relu")
)

# Output layer
nn.add(
    tf.keras.layers.Dense(units=1, activation="sigmoid")
)


# Check the structure of the model
nn.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


Looking at our model summary, we can see that the number of weight parameters (weight coefficients) for each layer equals the number of input values times the number of neurons plus a bias term for each neuron. Our first layer has 55 input values, and multiplied by the eight neurons (plus eight bias terms for each neuron) gives us a total of 448 weight parameters—plenty of opportunities for our model to find trends in the dataset. If there are eight neurons in the first layer, and five neurons in the second layer, then there are 45 parameters; eight times five equals 40 parameters, plus five parameters for the bias terms.

Now it is time to compile our model and define the loss and accuracy metrics. Since we want to use our model as a binary classifier, we'll use the `binary_crossentropy` loss function, `adam` optimizer, and `accuracy` metrics, which are the same parameters we used for our basic neural network. 

In [25]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [26]:
#train the model
fit_model = nn.fit(X_train, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 2s 3ms/step - loss: 378.6289 - accuracy: 0.8276
Epoch 2/100
35/35 [==============================] - 0s 2ms/step - loss: 209.3489 - accuracy: 0.8249
Epoch 3/100
35/35 [==============================] - 0s 3ms/step - loss: 85.2122 - accuracy: 0.8140
Epoch 4/100
35/35 [==============================] - 0s 2ms/step - loss: 25.7311 - accuracy: 0.7341
Epoch 5/100
35/35 [==============================] - 0s 2ms/step - loss: 19.7534 - accuracy: 0.7423
Epoch 6/100
35/35 [==============================] - 0s 2ms/step - loss: 16.4777 - accuracy: 0.7505
Epoch 7/100
35/35 [==============================] - 0s 2ms/step - loss: 13.7114 - accuracy: 0.7432
Epoch 8/100
35/35 [==============================] - 0s 3ms/step - loss: 10.3413 - accuracy: 0.7332
Epoch 9/100
35/35 [==============================] - 0s 2ms/step - loss: 7.6040 - accuracy: 0.7468
Epoch 10/100
35/35 [==============================] - 0s 2ms/step - loss: 4.9896 - accuracy: 0.7432

In [27]:
# evaluate model using test data
model_loss, model_accuracy = nn.evaluate(X_test, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 1.5313 - accuracy: 0.8696 - 331ms/epoch - 28ms/step
Loss: 1.5313377380371094, Accuracy: 0.8695651888847351


Looking at our deep learning model's performance metrics, the model was able to correctly identify employees who are at risk of attrition approximately 87% of the time. Considering that our input data included more than 30 different variables with more than 1,400 data points, the deep learning model was able to produce a fairly reliable classifier.